In [18]:
?SSlogis

SSlogis {stats},R Documentation
input,a numeric vector of values at which to evaluate the model.
Asym,a numeric parameter representing the asymptote.
xmid,a numeric parameter representing the x value at the inflection point of the curve. The value of SSlogis will be Asym/2 at xmid.
scal,a numeric scale parameter on the input axis.


In [67]:
phi_1 = 10
phi_2 = 2
phi_3 = 1

n_sample = 1000
epsilon = rnorm(n_sample, sd=1)

x = (runif(n_sample))*10  # [0, 100]
t = SSlogis(x, phi_1, phi_2, phi_3)+epsilon
t_norm = (t)/(max(t)-min(t))

dt = data.table(x=x, t=t, t_norm=t_norm)

# plot_ly(dt, x=~x, y=~t, type='scatter', mode='markers', name='t') %>%
#     add_trace(x=~x, y=~t_norm, type='scatter', name='t_norm')

In [68]:
model = nls(t~SSlogis(x, phi_1, phi_2, phi_3), data=dt)
model_norm = nls(t_norm~SSlogis(x, phi_1, phi_2, phi_3), data=dt)

y = predict(model, data=dt)
y_norm = predict(model_norm, data=dt)

coef(model)
coef(model_norm)

dt[, ':='(y=y, y_norm=y_norm)
    ][, .(r2 = 1-mean((t-y)^2)/var(t),
          r2_norm = 1-mean((t_norm-y_norm)^2)/var(t_norm))]



phi_1     phi_2     phi_3 
10.069625  1.970332  1.031029

phi_1     phi_2     phi_3 
0.8303985 1.7295050 1.1484590

r2,r2_norm
<dbl>,<dbl>
0.8885594,0.8880982
